## Chat z pamięcią

### Instalacja bibliotek

In [1]:
!pip install -q langchain-openai python-dotenv

### Utworzenie modelu

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

### History
History – przechowuje i wstrzykuje do promptu całą historię rozmowy wiadomość po wiadomości.

In [3]:
from langchain.memory import ChatMessageHistory

# message history
history = ChatMessageHistory()

history.add_user_message("Buenos dias!")
history.add_ai_message("hello!")
history.add_user_message("Whats your name?")
history.add_ai_message("My name is GIGACHAT")

# history.messages

In [4]:
history.messages

[HumanMessage(content='Buenos dias!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='hello!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Whats your name?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='My name is GIGACHAT', additional_kwargs={}, response_metadata={})]

### Memory

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory


llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

#typy wiadomości: user, assistant, system, function, tool
prompt = ChatPromptTemplate.from_messages([
    ("system", "Prowadzisz przyjazną rozmowę i pamiętasz kontekst. "
               "Zawsze odpowiadaj po polsku."),
    MessagesPlaceholder("history"),
    ("user", "{input}"),
])

chain = prompt | llm

store = {}
def get_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

chain_with_memory = RunnableWithMessageHistory(
    chain,
    get_session_history=get_history,
    input_messages_key="input",
    history_messages_key="history",
)

sid = "demo-session-123"

resp1 = chain_with_memory.invoke(
    {"input": "Cześć! Mam na imię Michał."},
    config={"configurable": {"session_id": sid}}
)
print(resp1.content)

resp2 = chain_with_memory.invoke(
    {"input": "Jak mam na imię?"},
    config={"configurable": {"session_id": sid}}
)
print(resp2.content)


Cześć, Michał! Miło mi cię poznać. Jak mija twój dzień?
Masz na imię Michał. Jakie masz plany na dzisiaj?


### Summary
Summary (memory) – zamiast pełnej historii przekazuje do modelu skondensowane podsumowanie wcześniejszych rozmów, co oszczędza tokeny i ułatwia skalowanie długich dialogów.

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

summarizer_prompt = ChatPromptTemplate.from_messages([
    ("system", "Summarize the following conversation briefly:"),
    ("human", "{conversation}")
])
summarizer = summarizer_prompt | llm | StrOutputParser()

conversation_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Here is the summary of prior conversation:\n{summary}"),
    MessagesPlaceholder("history"),
    ("human", "{input}")
])
conversation_chain = conversation_prompt | llm | StrOutputParser()

store = {}
summaries = {}

def get_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

def get_summary(session_id: str) -> str:
    if session_id not in summaries:
        summaries[session_id] = "No previous conversation."
    return summaries[session_id]

def update_summary(session_id: str, threshold: int = 4):
    """Update summary and clear old messages when threshold is reached"""
    history = get_history(session_id)

    if len(history.messages) >= threshold:
        current_summary = summaries.get(session_id, "")

        if current_summary and current_summary != "No previous conversation.":
            conversation_text = f"Previous summary: {current_summary}\n\nRecent messages: {history.messages}"
        else:
            conversation_text = str(history.messages)

        new_summary = summarizer.invoke({"conversation": conversation_text})
        summaries[session_id] = new_summary

        history.clear()

chain_with_memory = RunnableWithMessageHistory(
    conversation_chain,
    get_session_history=get_history,
    input_messages_key="input",
    history_messages_key="history"
)

sid = "demo-summary"

cfg = {"configurable": {"session_id": sid}}
summary = get_summary(sid)
response1 = chain_with_memory.invoke({"input": "Cześć! Mam na imię Michał.", "summary": summary}, cfg)
print(response1)

# Update summary periodically (e.g., after every 2-3 exchanges)
update_summary(sid, threshold=4)

summary = get_summary(sid)
response2 = chain_with_memory.invoke({"input": "Jak mam na imię?", "summary": summary}, cfg)
print(response2)

update_summary(sid, threshold=4)


Cześć, Michał! Jak mogę Ci dzisiaj pomóc?
Masz na imię Michał. Jakie masz pytania lub o czym chciałbyś porozmawiać?
